# QP-IK
This tutorial aims to give the user a receipt that can be followed to design inverse kinematics exploiting `bipedal-locomotion-framework`. 

In this tutorial, we will control the end-effector of the `iiwa robot` to perform a predefined trajectory.

## Import all the required packages
In this section we import all the required packages, in detail, we need `bipedal_locomotion_framework` to solve the IK problem. `robot_descriptions` is required to load the model of the `iiwa`. We exploit `iDynTree` for visualization and to execute dynamics algorithms. Finally `manifpy` is in charge handle rotation matrices and homogeneous transformations.

⚠️ Please note that `load_robot_description` can be also used to load the iCub humanoid robot, however, we sudgest using the official model stored in [`icub-models`](https://github.com/robotology/icub-models).

In [ ]:
import bipedal_locomotion_framework as blf
from robot_descriptions.loaders.idyntree import load_robot_description
from idyntree.visualize import MeshcatVisualizer
import idyntree.bindings as idyn
import numpy as np
import manifpy as manif
import datetime

## Initialize the script
We now load the robot model and pass it to the `iDynTree.KinDynComputations` object. 
The model has loaded thanks to `load_robot_description` and passed to `KinDynComputations` thanks to `loadRobotModel()`. Once the model is loaded we set the initial joint configuration.

In [ ]:
dt = 0.01
# convert the period in milliseconds
period = datetime.timedelta(seconds=dt)

robot = load_robot_description("iiwa_description")
initial_config = [0, -np.pi/4, 0, np.pi/2, 0, -np.pi/4, 0]
kindyn = idyn.KinDynComputations()
kindyn.loadRobotModel(robot)
kindyn.setJointPos(initial_config)

We define the `Simulator` class that is useful to handle the integration of the dynamical system. You can set the control input (the joint velocity) perform a simulation step and retrieve the output (the joint position)

In [ ]:
class Simulator:
    def __init__(self, initial_config, dt):
        self.system = blf.continuous_dynamical_system.FloatingBaseSystemKinematics()
        self.system.set_state(([0,0,0],  manif.SO3.Identity(), initial_config))
        
        self.integrator = blf.continuous_dynamical_system.FloatingBaseSystemKinematicsForwardEulerIntegrator()
        self.integrator.set_dynamical_system(self.system)
        assert self.integrator.set_integration_step(dt)
        self.dt = dt
        self.zero = datetime.timedelta(milliseconds=0)
    
    def set_control_input(self, joint_velocity):
        self.system.set_control_input(([0,0,0,0,0,0] , joint_velocity))
        
    def integrate(self):
        self.integrator.integrate(self.zero, self.dt)

We now get the initial position of the end effector, this will be useful to generate the reference end-effector trajectory. We also initialize the `Simulator`.

In [ ]:
ee_transform_0 = kindyn.getWorldTransform("iiwa_link_ee")
ee_orientation_0 = ee_transform_0.getRotation().toNumPy()
ee_position_0 = ee_transform_0.getPosition().toNumPy()

simulator = Simulator(initial_config=initial_config, dt=period)

We now construct the IK problem. To do so we read the parameters stored in `./config/ik.toml`. This file contains the description of all the tasks required by the solver. Then thanks to `blf.utils.create_ik` we build the IK problem. `blf.utils.create_ik` return the solver and a dictionary containing the tasks. By accessing a task you can set the desired setpoint.

In [ ]:
params = blf.parameters_handler.TomlParametersHandler()
assert params.set_from_file("./config/ik.toml")
_, _, solver = blf.ik.QPInverseKinematics.build(kin_dyn = kindyn, param_handler = params);

We now initialize the visualizer. We add the model of the robot and a sphere showing the desired position of the end effector. You can open the visualizer by visiting the `URL` provided.

In [ ]:
viz = MeshcatVisualizer()
viz.load_model(robot, model_name="iiwa", color=0.8)
viz.load_sphere(radius=0.02, shape_name="goal", color=[1, 0.6, 0, 1])
viz.set_primitive_geometry_transform(position=ee_position_0, rotation=np.eye(3), shape_name="goal")
viz.set_multibody_system_state([0,0,0], np.eye(3), initial_config, model_name="iiwa")
viz.jupyter_cell()

We finally design the trajectory that should be followed by the end effector.

In [ ]:
def trajectory(time):
    amplitude = 0.5
    freq = 0.1
    pos = np.array([0, amplitude * np.sin(2 * np.pi * freq * time), 0 ]) + ee_position_0
    vel = np.array([0, 2 * np.pi * freq * amplitude * np.cos(2 * np.pi * freq * time), 0 ])

    return pos, vel

## Main control loop

This section contains the main control loop. In detail, we first set the task for the base position (In this case the base is set in the identity and its velocity is set to zero). Moreover to handle redundancy we also set the `joint regularization task` as the initial config.  

Then we perform the following action in the loop:
1. retrieve the desired trajectory;
2. solve the IK problem;
3. propagate the simulation;
4. update the kinDynComputations object;
5. update the visualization.

In [ ]:
solver.get_task("BASE_TASK").set_set_point(manif.SE3.Identity(), manif.SE3Tangent.Zero())
solver.get_task("JOINT_REGULARIZATION_TASK").set_set_point(initial_config)

for i in range(1000):

    tic = blf.clock().now()
    
    # retrieve the desired trajectory
    pos, vel = trajectory(i * dt)
    spatial_vel = np.hstack([vel, np.array([0,0,0])])
    ee_transform_des = blf.conversions.to_manif_pose(ee_orientation_0, pos)
    solver.get_task("EE_TASK").set_set_point(ee_transform_des, manif.SE3Tangent(spatial_vel))
    
    # solve the IK problem
    if not solver.advance():
        raise ValueError("Unable to solve the IK problem.")
    
    # propagate the simulation
    simulator.set_control_input(solver.get_output().joint_velocity)
    simulator.integrate()
    base_position, base_rotation, joint_position = simulator.integrator.get_solution()
    
    # update the kinDynComputations object
    kindyn.setJointPos(joint_position)
    
    # update the visualization
    viz.set_primitive_geometry_transform(position=pos, rotation=np.eye(3), shape_name="goal")
    viz.set_multibody_system_state(base_position, base_rotation.rotation(), joint_position, model_name="iiwa")
    
    # Sleep if required
    toc = blf.clock().now()
    delta_time = toc - tic
    if delta_time < period:
        blf.clock().sleep_for(period - delta_time)